In [13]:
import math
import random
from itertools import chain

global k_fold

sc = SparkContext.getOrCreate()

TRAIN_TEST = 0.8
k_fold=10

global n_columns
global w
global b
global n_row
global y_h_m_y


n_columns=56 #total number of columns (58) - the one deleted (57) - label (56)
col_sums=[]
averages=[]
sigmas=[]
w=[]



def initializeAccumulators():
    i=0
    while(i<n_columns):
        averages.append(sc.accumulator(0))
        col_sums.append(sc.accumulator(0))
        sigmas.append(sc.accumulator(0)) 
        i+=1

def addToAccumulators(row):
    if(len(row)!=len(col_sums)):
        raise Exception("Number of columns in the row doesn't mach the number of accumulators initiated. Len row: "+str(len(row))+" n_accomulators"+str(len(col_sums)))
    i=0
    while(i<n_columns):
        print("Len row: "+str(len(row))+" n_accomulators"+str(len(col_sums)))
        col_sums[i].add(row[i])
        i+=1
        
def preprocessing(row):
    #splitting
    split=row.split(" ")
    label = int(split[57])
    del split[57] #label
    del split[56] #col 57th
    split=[float(col) for col in split]
    addToAccumulators(split)
    #assign random key to the datapoint
    key=random.getrandbits(64)
    #assign train/test 
    if(random.random()<TRAIN_TEST):
        train=1
    else:
        train=0    
    #return
    return (key,train,split,label)
def calcAvg(n_row):
    i=0
    while(i<n_columns):
        averages[i]=col_sums[i].value/n_row
        i+=1
def calcResiduals(row):
    i=0
    while(i<n_columns):       
        sigmas[i].add(math.pow(row[2][i]-averages[i],2))
        i+=1
    return row
def calcSigmas(n_row):
    i=0
    while(i<n_columns):
        sigmas[i]=math.sqrt(sigmas[i].value/float(n_row-1))
        i+=1
def normalize(row):
    i=0
    while(i<n_columns):
        row[2][i]=(row[2][i]-averages[i])/sigmas[i]
        i+=1
    return row

def initializeWeights(random_init=False):
    if(random_init):
        #return sc.parallelize([(i, random.random()) for i in range(0,n_columns)])
        return [random.random() for i in range(0,n_columns)]
    else:
        #return sc.parallelize([(i, 0.0) for i in range(0,n_columns)])
        return [0.0 for i in range(0,n_columns)]

def initializeBias():
    return sc.parallelize(0.0)

def sigmoid(z):
    return 1/(1+math.exp(-z))

def predict(w,b,X):
    return sigmoid(sum([X[i]*w[i] for i in range(len(w))])+b)

def predict_parallel(w,b,X):
    #Change X representation
    X = sc.parallelize(X).zipWithIndex().map(lambda x: (x[1],x[0]))
    wX_plus_b=X.join(w).map(lambda x: (x[1][0]*x[1][1])+b).sum()
    return sigmoid(wX_plus_b)

def compute_cost(dataset,w,b,lambda_reg):
    return (-1/dataset.count())*dataset.map(lambda x: x[3]*math.log(predict(w,b,x[2])) \
                                    +(1-x[3])*math.log(1-predict(w,b,x[2]))).sum() \
                                    + lambda_reg/(2*dataset.count())*sum([i*i for i in w]);

def make_folds(x):
    return (x[0],int(random.random()*10-1),x[1],x[2])

def gradientDescent(iterations,train_fold,w,train_fold_size,lambda_reg,learning_rate,b):
    costs=[]
    for iteration in range(iterations):
        w,b = gradientDescentIteration(train_fold,w,train_fold_size,lambda_reg,learning_rate,b)
        cost=compute_cost(train_fold,w,b,lambda_reg)
        costs.append(cost)
        print("-> Iteration done: "+str(iteration+1)+" of "+str(iterations)+". Cost: "+str(cost))
    return w,b,costs
    

def gradientDescentIteration(train_fold,w,train_fold_size,lambda_reg,learning_rate,b):
    dw=[i for i in range(0,n_columns)]
    for j in range(n_columns):
        X_j=train_fold.map(lambda x: (predict(w,b,x[2])-x[3])*x[2][j]).sum()
        dw[j]=(1/train_fold_size)*X_j+(lambda_reg/train_fold_size)+w[j]
        w[j]-=learning_rate*dw[j]
    b-=learning_rate*(1/train_fold_size)*train_fold.map(lambda x: predict(w,b,x[2])-x[3]).sum()
    return w,b

def kFoldsCV(k_fold,iterations,train,lambda_reg,learning_rate):
    fold_length=train.count()/k_fold
    costs_fold=[]
    for i_fold in range(k_fold):
        starting_fold=fold_length*i_fold
        end_fold=starting_fold+fold_length
        test_fold=train.zipWithIndex().filter(lambda t: (t[1]>=starting_fold and t[1]<end_fold)).map(lambda t: t[0]) #the map get rid of the index again
        train_fold=train.zipWithIndex().filter(lambda t: t[1]<starting_fold or t[1]>=end_fold).map(lambda t: t[0])
        train_fold_size=train_fold.count()
        w=initializeWeights()
        b=0
        #Gradient descent
        costs,w,b = gradientDescent(iterations,train_fold,w,train_fold_size,lambda_reg,learning_rate,b)

        costs_fold.append(costs)
        print("--> Fold #"+str(i_fold+1)+" of "+str(k_fold)+" is done. Average error: "+str(sum(costs_fold)/iterations))
    return w,b,costs_fold

def trainTestSplit(dataset):
    dataset=dataset.map(normalize)
    train=dataset.filter(lambda x: x[1]==1)
    train_size=train.count()
    test=dataset.filter(lambda x: x[1]==0)
    return train, test

def train(filename, iterations, learning_rate, lambda_reg):
    global w
    global b
    initializeAccumulators()
    dataset=sc.textFile(filename).map(preprocessing).sortBy(lambda x: x[0])
    n_row=dataset.count()
    calcAvg(n_row)
    dataset.map(calcResiduals).collect()
    calcSigmas(n_row)
    train, test = trainTestSplit(dataset)
    print("Split train/test done. Train contains "+str(train.count())+" elements, Test contains "+str(test.count())+" elements")

    w, b, costs_fold = kFoldsCV(k_fold,iterations,train,lambda_reg,learning_rate)

    costs_fold=list(chain.from_iterable(costs_fold))
    print("---> "+str(k_fold)+"-fold validation done. Average error: "+str(sum(costs_fold)/(iterations*k_fold)))
            
grid=[]
grid+=[[(i*0.001,i*0.002),(i*0.002,i*0.001)] for i in range(1,10)]
grid=list(chain.from_iterable(grid))
for par in grid:
    train("dataset/spam.data",15,par[0],par[1])

[(0.001, 0.002), (0.002, 0.001), (0.002, 0.004), (0.004, 0.002), (0.003, 0.006), (0.006, 0.003), (0.004, 0.008), (0.008, 0.004), (0.005, 0.01), (0.01, 0.005), (0.006, 0.012), (0.012, 0.006), (0.007, 0.014), (0.014, 0.007), (0.008, 0.016), (0.016, 0.008), (0.009000000000000001, 0.018000000000000002), (0.018000000000000002, 0.009000000000000001)]
Split train/test done. Train contains 3674 elements, Test contains 927 elements
-> Iteration done: 1 of 15. Cost: 0.6925793736402749
-> Iteration done: 2 of 15. Cost: 0.6920146042986751
-> Iteration done: 3 of 15. Cost: 0.6914528507733275
-> Iteration done: 4 of 15. Cost: 0.690894091472674
-> Iteration done: 5 of 15. Cost: 0.6903383049741655
-> Iteration done: 6 of 15. Cost: 0.6897854700230417
-> Iteration done: 7 of 15. Cost: 0.6892355655311032
-> Iteration done: 8 of 15. Cost: 0.688688570575494
-> Iteration done: 9 of 15. Cost: 0.6881444643974777
-> Iteration done: 10 of 15. Cost: 0.6876032264012427
-> Iteration done: 11 of 15. Cost: 0.6870648

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [ ]:

grid